In [ ]:
import os
import ast
import re
import json
from IPython.display import display, HTML
import numpy as np
import pandas as pd
from pydantic import BaseModel
import json
import time

import torch

import seaborn as sns
import matplotlib.pyplot as plt
import torch
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold

In [ ]:
# Load the JSON file
with open("/content/job_listings.json", "r", encoding="utf-8") as f:
    job_data = json.load(f)

# Import Data
df_x = pd.read_csv('/content/x_train_Meacfjr.csv')
df_y = pd.read_csv('/content/y_train_SwJNMSu.csv')

# Job embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
job_embeddings = torch.load('/content/job_embeddings_small_norm.pth', map_location=device)

<ipython-input-2-de8b1349da62>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  job_embeddings = torch.load('/content/job_embeddings_small_norm.pth', map_location=device)


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# # # Load the JSON file
# with open("/content/drive/MyDrive/Colab Notebooks/Challenge/data/job_listings.json", "r") as file:
#     job_data = json.load(file)


# df_x = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Challenge/data/x_train_Meacfjr.csv", delimiter=",", quotechar='"')
# df_y = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Challenge/data/y_train_SwJNMSu.csv", delimiter=",", quotechar='"')

# # Job Embeddings
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# embedding_path = "/content/drive/MyDrive/Colab Notebooks/Challenge/data/job_embeddings_small.pth"
# print("Loading embeddings from {}".format(embedding_path))
# job_embeddings = torch.load(embedding_path, map_location=device)

Mounted at /content/drive
Loading embeddings from /content/drive/MyDrive/Colab Notebooks/Challenge/data/job_embeddings_small.pth


<ipython-input-3-766ebfb8b1a0>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  job_embeddings = torch.load(embedding_path, map_location=device)


In [ ]:
# Check the first few keys
# print(list(job_data.keys())[:5])

# Print a sample entry
# sample_key = list(job_data.keys())[0]
# print(job_data[sample_key])


In [ ]:

# ============================================================
# 0. Load Data
# ============================================================

# Convert string representation of lists into actual lists
df_x["job_ids"] = df_x["job_ids"].apply(ast.literal_eval)
df_x["actions"] = df_x["actions"].apply(ast.literal_eval)

#print df
# print(df_x)
# print(df_y)

In [ ]:
def expand_dataset(x, y):
    y_indexed = y.set_index('session_id')

    new_df_x_rows = []
    new_df_y_rows = []
    new_session_id = 0

    for _, row in x.iterrows():
        orig_session = row['session_id']
        job_ids = row['job_ids']
        actions = row['actions']

        # Preserve numerical features from the original row
        total = row.get('total', 0)  # Ensure default value if missing
        apply_count = row.get('apply_count', 0)
        view_count = row.get('view_count', 0)

        # Get target data
        target_row = y_indexed.loc[orig_session]
        target_job = target_row['job_id']
        target_action = target_row['action']

        n = len(job_ids)
        for i in range(n):
            prefix_jobs = job_ids[:i+1]
            prefix_actions = actions[:i+1]

            if i < n - 1:
                t_job = job_ids[i+1]
                t_action = actions[i+1]
            else:
                t_job = target_job
                t_action = target_action

            new_df_x_rows.append({
                'session_id': new_session_id,
                'job_ids': prefix_jobs,
                'actions': prefix_actions,
                'total': total,  # Keep original features
                'apply_count': apply_count,
                'view_count': view_count
            })
            new_df_y_rows.append({
                'session_id': new_session_id,
                'job_id': t_job,
                'action': t_action
            })
            new_session_id += 1

    return pd.DataFrame(new_df_x_rows), pd.DataFrame(new_df_y_rows)


## Model for feature weights learning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class JobRecommendationModel(nn.Module):
    def __init__(self, job_embeddings, embedding_dim, device):
        super().__init__()
        self.job_embeddings = {k: v.to(device) for k, v in job_embeddings.items()}  # Move embeddings to GPU
        self.embedding_dim = embedding_dim
        self.device = device
        self.feature_weights = nn.Parameter(torch.ones(len(job_embeddings)).to(device))  # Learnable weights

    def compute_user_preference(self, job_ids):
        """Computes weighted user preference embedding."""
        job_embeds = []
        for feature, embeddings in self.job_embeddings.items():
            job_embeds.append(embeddings[job_ids])  # Get embeddings for the given jobs
        job_embeds = torch.stack(job_embeds, dim=0)  # Shape: (num_features, num_jobs, embedding_dim)
        weighted_embeds = self.feature_weights.view(-1, 1, 1) * job_embeds  # Apply weights
        user_pref = weighted_embeds.mean(dim=1).sum(dim=0)  # Mean over jobs, sum over features
        return user_pref / self.feature_weights.sum()  # Normalize

    def forward(self, user_jobs, positive_job, negative_job):
        """Computes BPR loss for training."""
        user_pref = self.compute_user_preference(user_jobs)  # User embedding
        # Ensure all embeddings are 2D (batch_size=1)
        user_pref = user_pref.unsqueeze(0)  # Shape: [1, embedding_dim]
        pos_emb = pos_emb.unsqueeze(0)      # Shape: [1, embedding_dim]
        neg_emb = neg_emb.unsqueeze(0)      # Shape: [1, embedding_dim]

        print(user_pref.shape)
        print(self.compute_user_preference(positive_job).shape)
        pos_sim = torch.cosine_similarity(user_pref, self.compute_user_preference(positive_job))
        neg_sim = torch.cosine_similarity(user_pref, self.compute_user_preference(negative_job))
        return -torch.log(torch.sigmoid(pos_sim - neg_sim)).mean()  # BPR loss

In [ ]:
def train_model(job_embeddings, x_train, y_train, num_epochs=10, lr=0.01, device='cuda'):
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = JobRecommendationModel(job_embeddings, embedding_dim=job_embeddings[list(job_embeddings.keys())[0]].shape[1], device=device).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        total_loss = 0.0
        for _, session in x_train.groupby('session_id'):
            job_ids = torch.tensor(session['job_ids'].values[0], dtype=torch.long, device=device)
            positive_job = torch.tensor(y_train[y_train['session_id'] == session['session_id'].iloc[0]]['job_id'].values, dtype=torch.long, device=device)
            negative_job = torch.randint(0, job_embeddings[list(job_embeddings.keys())[0]].shape[0], (positive_job.shape[0],), device=device)  # Random negatives

            loss = model(job_ids, positive_job, negative_job)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(x_train)}")
    return model

In [ ]:
trained_model = train_model(job_embeddings, df_x, df_y)

UnboundLocalError: cannot access local variable 'pos_emb' where it is not associated with a value

## First Version


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score

In [ ]:
# ---------------------------
# 1. Load Job Embeddings (Using GPU)
# ---------------------------
feature_weights = {
    'title_section': 1.0,
    'location': 0.8,
    'seniority': 0.8,
    'company': 0.5,
    'industry': 0.5
    }
action_weights = {'apply': 10, 'view': 1}

weighted_features = [feature_weights[feature] * job_embeddings[feature] for feature in feature_weights.keys()]
job_embedding_matrix = torch.cat(weighted_features, dim=1).to(device) # job_embedding_matrix.shape = (n_jobs, n_features*emb_dim)

In [ ]:
print(job_embeddings['title_section'].shape)
print(len(weighted_features))
print(job_embedding_matrix.shape)

torch.Size([21917, 1024])
5
torch.Size([21917, 5120])


In [ ]:
# ---------------------------
# 2. Build Interaction Matrix from df_x
# ---------------------------
def build_interaction_matrix(x_train, weight_map={'apply': 5, 'view': 1}):
    sessions = x_train['session_id'].unique()
    all_jobs = set()
    for jobs in x_train['job_ids']:
        all_jobs.update(jobs)
    all_jobs = list(all_jobs)

    session2idx = {s: i for i, s in enumerate(sessions)}
    job2idx = {j: i for i, j in enumerate(all_jobs)}
    # job2idx = {row["job_id"]: idx for idx, row in x_train.iterrows()}

    matrix = np.zeros((len(sessions), len(all_jobs)))
    for _, row in x_train.iterrows():
        s_idx = session2idx[row['session_id']]
        for job, act in zip(row['job_ids'], row['actions']):
            j_idx = job2idx[job]
            matrix[s_idx, j_idx] += weight_map.get(act,1)

    return matrix, session2idx, job2idx

#interaction_matrix, session2idx, _ = build_interaction_matrix(df_x) # interaction_matrix.shape = (n_sessions , n_jobs)
interaction_matrix, session2idx, job2idx = build_interaction_matrix(df_x) # interaction_matrix.shape = (n_sessions , n_jobs)

In [ ]:
# ---------------------------
# Compute Session Features for Action Prediction from df_x
# ---------------------------
# (Assuming each row in df_x has a list of actions corresponding to the jobs in that session)
df_x['apply_count'] = df_x['actions'].apply(lambda acts: sum(1 for act in acts if act == 'apply'))
df_x['view_count'] = df_x['actions'].apply(lambda acts: sum(1 for act in acts if act == 'view'))
df_x['total'] = df_x['apply_count'] + df_x['view_count']

session_features = df_x.groupby('session_id').agg({
    'total': 'sum',
    'apply_count': 'sum',
    'view_count': 'sum'
}).reset_index()

In [ ]:
# ---------------------------
# 3. Align Job Embeddings (Using GPU)
# ---------------------------
# Create a mapping from job id (as string) to the row index in the job_embedding_matrix

def build_aligned_embeddings(job2idx, job_embedding_matrix):


    embedding_map = {str(idx): idx for idx in range(job_embedding_matrix.shape[0])}
    num_jobs = len(job2idx)
    emb_dim = job_embedding_matrix.shape[1]
    aligned_embeddings = torch.zeros((num_jobs, emb_dim), device=device)

    available_embeddings = []
    for job, idx in job2idx.items():
        job_str = str(job)
        if job_str in embedding_map:
            emb = job_embedding_matrix[embedding_map[job_str]]
            aligned_embeddings[idx] = emb
            available_embeddings.append(emb)

    if available_embeddings:
        avg_embedding = torch.stack(available_embeddings).mean(dim=0)
        missing = torch.where(aligned_embeddings.norm(dim=1) == 0)[0]
        aligned_embeddings[missing] = avg_embedding

    return aligned_embeddings

aligned_embeddings = build_aligned_embeddings(job2idx, job_embedding_matrix)
#aligned_embeddings = build_aligned_embeddings(job2idx, job_emb_matrix_pca)
#aligned_embeddings = job_embedding_matrix

In [ ]:
# ---------------------------
# 4. Compute User Factors Using SVD (Collaborative Filtering)
# ---------------------------
n_components = 200
svd = TruncatedSVD(n_components=n_components, random_state=42)
user_factors = svd.fit_transform(interaction_matrix)  # Shape: (num_sessions, n_components)
job_factors = svd.components_.T  # Shape: (num_jobs, n_components)

In [ ]:
# ---------------------------
# 5. Hybrid Recommendation with GPU-Based Content Filtering
# ---------------------------
def hybrid_recommendation(session_idx, user_factors, job_factors, interaction_matrix, aligned_embeddings, alpha=0.7, top_n=10):
    """Hybrid CF + GPU-based CBF recommendation system."""

    # Get Collaborative Filtering (CF) Scores
    cf_scores = user_factors[session_idx] @ job_factors.T  # Shape: (num_jobs,)
    cf_scores = (cf_scores - cf_scores.min()) / (cf_scores.max() - cf_scores.min() + 1e-9)
    #cf_scores = torch.full((aligned_embeddings.shape[0],), cf_scores.mean(), device=device)


    # Build session embedding (weighted job embeddings) from the interaction matrix
    interacted_jobs = np.where(interaction_matrix[session_idx] > 0)[0]  # Indices of jobs the user interacted with

    if len(interacted_jobs) > 0:
        session_embeddings = aligned_embeddings[interacted_jobs]  # Shape: (num_interacted_jobs, emb_dim)
        action_weights = torch.tensor(interaction_matrix[session_idx, interacted_jobs], device=device).float()
        action_weights = action_weights.view(-1, 1)
        user_profile_vector = torch.sum(session_embeddings * action_weights, dim=0) / torch.sum(action_weights)
    else:
        user_profile_vector = torch.mean(aligned_embeddings, dim=0)  # Fallback if no interactions

    # Compute Content-Based Filtering (CBF) Scores using cosine similarity on GPU
    cbf_scores = F.cosine_similarity(user_profile_vector.unsqueeze(0), aligned_embeddings, dim=1)  # (num_jobs,)

    # Final Hybrid Score: blend CF and CBF scores using alpha
    final_scores = alpha * torch.tensor(cf_scores, device=device) + (1 - alpha) * cbf_scores

    # Exclude already interacted jobs from recommendations
    final_scores[torch.tensor(interaction_matrix[session_idx], device=device) > 0] = float('-inf')

    # Get top-N recommended job indices
    top_indices = torch.argsort(final_scores, descending=True)[:top_n].cpu().numpy()
    return top_indices

In [ ]:
# ---------------------------
# 5. Hybrid Recommendation with GPU-Based Content Filtering
# ---------------------------
def hybrid_recommendation(session_idx, user_factors, job_factors, interaction_matrix, aligned_embeddings, alpha=0.7, top_n=10):
    """Hybrid CF + GPU-based CBF recommendation system."""

    # Get Collaborative Filtering (CF) Scores
    cf_scores = user_factors[session_idx] @ job_factors.T  # Shape: (num_jobs,)
    cf_scores = (cf_scores - cf_scores.min()) / (cf_scores.max() - cf_scores.min() + 1e-9)
    cf_scores = torch.full((aligned_embeddings.shape[0],), 0, device=device)



    # Build session embedding (weighted job embeddings) from the interaction matrix
    interacted_jobs = np.where(interaction_matrix[session_idx] > 0)[0]  # Indices of jobs the user interacted with

    if len(interacted_jobs) > 0:
        session_embeddings = aligned_embeddings[interacted_jobs]  # Shape: (num_interacted_jobs, emb_dim)
        action_weights = torch.tensor(interaction_matrix[session_idx, interacted_jobs], device=device).float()
        action_weights = action_weights.view(-1, 1)
        user_profile_vector = torch.sum(session_embeddings * action_weights, dim=0) / torch.sum(action_weights)
    else:
        user_profile_vector = torch.mean(aligned_embeddings, dim=0)  # Fallback if no interactions

    # Compute Content-Based Filtering (CBF) Scores using cosine similarity on GPU
    cbf_scores = F.cosine_similarity(user_profile_vector.unsqueeze(0), aligned_embeddings, dim=1)  # (num_jobs,)

    # Final Hybrid Score: blend CF and CBF scores using alpha
    final_scores = alpha * cf_scores.clone().detach().float() + (1 - alpha) * cbf_scores

    # Exclude already interacted jobs from recommendations
    # Create a mask with the correct size for final_scores
    mask = torch.zeros_like(final_scores, dtype=torch.bool, device=device)
    # Map interacted job indices to the corresponding indices in final_scores
    interacted_job_indices_in_final_scores = [job2idx[job_id] for job_id in df_x.loc[df_x['session_id'] == s_id, 'job_ids'].iloc[0] if job_id in job2idx]
    mask[interacted_job_indices_in_final_scores] = True
    final_scores[mask] = float('-inf')

    # Get top-N recommended job indices
    top_indices = torch.argsort(final_scores, descending=True)[:top_n].cpu().numpy()
    return top_indices

In [ ]:
# ---------------------------
# 6. Recommendation Pipeline & Evaluation (GPU-Based)
# ---------------------------
results = {}
y_true_jobs = []
y_pred_jobs = []
y_true_actions = []
y_pred_actions = []
idx2job = {i: j for j, i in job2idx.items()}

for _, row in df_y.iterrows():
    s_id = row['session_id']
    if s_id in session2idx:
        s_idx = session2idx[s_id]

        # Get top-N job recommendations for this session
        top_indices = hybrid_recommendation(s_idx, user_factors, job_factors, interaction_matrix, aligned_embeddings, alpha=0.5, top_n=10)
        recommended_jobs = [i for i in top_indices]

        # Save the ground truth and predicted job recommendations
        y_true_jobs.append(row['job_id'])
        y_pred_jobs.append(recommended_jobs)

        # Use session features from df_x to predict action (since df_y only contains one action per session)
        feat = session_features[session_features['session_id'] == s_id][['total', 'apply_count', 'view_count']]
        if not feat.empty:
            predicted_action = 'apply' if feat['apply_count'].sum() > feat['view_count'].sum() else 'view'
        else:
            predicted_action = 'view'

        y_true_actions.append(row['action'])
        y_pred_actions.append(predicted_action)

In [ ]:

# ---------------------------
# 7. Evaluation Metrics
# ---------------------------
def mean_reciprocal_rank(y_true, y_pred):
    scores = []
    for true_job, ranked_jobs in zip(y_true, y_pred):
        if true_job in ranked_jobs:
            rank = ranked_jobs.index(true_job) + 1
            scores.append(1 / rank)
        else:
            scores.append(0)
    return sum(scores) / len(scores) if scores else 0

mrr_score = mean_reciprocal_rank(y_true_jobs, y_pred_jobs)
action_accuracy = accuracy_score(
    [1 if x == 'apply' else 0 for x in y_true_actions],
    [1 if x == 'apply' else 0 for x in y_pred_actions]
)
final_score = (0.7 * mrr_score) + (0.3 * action_accuracy)

print("\nEvaluation Results:")
print(f"  🔹 MRR Score: {mrr_score:.4f}")
print(f"  🔹 Action Prediction Accuracy: {action_accuracy:.4f}")
print(f"  🔹 Final Score (70% MRR + 30% Accuracy): {final_score:.4f}")


Evaluation Results:
  🔹 MRR Score: 0.0006
  🔹 Action Prediction Accuracy: 0.7766
  🔹 Final Score (70% MRR + 30% Accuracy): 0.2334


## Second Version

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
# ---------------------------
# 3. Compute Job Popularity
# ---------------------------
job_popularity = np.sum(interaction_matrix > 0, axis=0)
popularity_penalty = 1 / (job_popularity + 1)  # Inverse frequency


In [ ]:
# ---------------------------
# 4. Compute User Factors Using SVD (Collaborative Filtering)
# ---------------------------
n_components = 200  # Increased from 100
svd = TruncatedSVD(n_components=n_components, random_state=42)
user_factors = svd.fit_transform(interaction_matrix)
job_factors = svd.components_.T

# Normalize CF scores
user_factors = (user_factors - user_factors.min()) / (user_factors.max() - user_factors.min() + 1e-9)

In [ ]:
# ---------------------------
# 5. Align Job Embeddings (Using GPU)
# ---------------------------
def build_aligned_embeddings(job2idx, job_embedding_matrix):
    num_jobs = len(job2idx)
    emb_dim = job_embedding_matrix.shape[1]
    aligned_embeddings = torch.zeros((num_jobs, emb_dim), device=device)

    for job, idx in job2idx.items():
        if job in job_embeddings:
            aligned_embeddings[idx] = job_embedding_matrix[job]

    avg_embedding = aligned_embeddings.mean(dim=0)
    missing = torch.where(aligned_embeddings.norm(dim=1) == 0)[0]
    aligned_embeddings[missing] = avg_embedding

    return aligned_embeddings

# aligned_embeddings = build_aligned_embeddings(job2idx, job_embedding_matrix)

In [ ]:
# ---------------------------
# 6. Hybrid Recommendation
# ---------------------------
def hybrid_recommendation(session_idx, user_factors, job_factors, interaction_matrix, aligned_embeddings, alpha=0.7, top_n=10):
    """Hybrid CF + GPU-based CBF recommendation system."""

    cf_scores = user_factors[session_idx] @ job_factors.T

    interacted_jobs = np.where(interaction_matrix[session_idx] > 0)[0]
    if len(interacted_jobs) > 0:
        session_embeddings = aligned_embeddings[interacted_jobs]
        action_weights = torch.tensor(interaction_matrix[session_idx, interacted_jobs], device=device).float()
        action_weights = action_weights.view(-1, 1)
        user_profile_vector = torch.sum(session_embeddings * action_weights, dim=0) / torch.sum(action_weights)
    else:
        user_profile_vector = aligned_embeddings.mean(dim=0)

    cbf_scores = user_profile_vector @ aligned_embeddings.T

    final_scores = alpha * torch.tensor(cf_scores, device=device) + (1 - alpha) * cbf_scores
    final_scores *= torch.tensor(popularity_penalty, device=device)

    final_scores[torch.tensor(interaction_matrix[session_idx], device=device) > 0] = float('-inf')

    top_indices = torch.argsort(final_scores, descending=True)[:top_n].cpu().numpy()
    return top_indices

In [ ]:
# ---------------------------
# MLP Model
# ---------------------------
class ActionPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(3, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

In [ ]:
# ---------------------------
# Cross-Validation Setup
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_mrr_scores = []
all_action_accuracies = []
best_model = None
best_score = float('-inf')

for fold, (train_idx, test_idx) in enumerate(kf.split(df_x)):
    print(f"\n Fold {fold+1} of Cross-Validation")

    # Split original dataset
    df_x_train, df_x_test = df_x.iloc[train_idx], df_x.iloc[test_idx]
    df_y_train, df_y_test = df_y.iloc[train_idx], df_y.iloc[test_idx]

    # Expand dataset
    df_x_train_expanded, df_y_train_expanded = expand_dataset(df_x_train, df_y_train)

    # Extract features and labels
    X_train = df_x_train_expanded[['total', 'apply_count', 'view_count']].values
    y_train = (df_x_train_expanded['apply_count'] > df_x_train_expanded['view_count']).astype(int).values
    X_test = df_x_test[['total', 'apply_count', 'view_count']].values
    y_test = (df_x_test['apply_count'] > df_x_test['view_count']).astype(int).values

    # Convert to tensors
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32).to(device), torch.tensor(y_train, dtype=torch.float32).to(device)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32).to(device), torch.tensor(y_test, dtype=torch.float32).to(device)

    # Initialize model, optimizer, loss function
    model = ActionPredictor().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()

    # Train the model
    for epoch in range(60):
        optimizer.zero_grad()
        y_pred = model(X_train).squeeze()
        loss = loss_fn(y_pred, y_train)
        loss.backward()
        optimizer.step()

    # Evaluate Action Prediction
    y_pred_test = model(X_test).squeeze().round().detach().cpu().numpy()
    action_accuracy = accuracy_score(y_test.cpu().numpy(), y_pred_test)

    # Compute MRR
    def mean_reciprocal_rank(y_true, y_pred):
        scores = []
        for true_job, ranked_jobs in zip(y_true, y_pred):
            if true_job in ranked_jobs:
                rank = ranked_jobs.index(true_job) + 1
                scores.append(1 / rank)
            else:
                scores.append(0)
        return sum(scores) / len(scores) if scores else 0

    mrr_score = mean_reciprocal_rank(y_true_jobs, y_pred_jobs)
    final_score = (0.7 * mrr_score) + (0.3 * action_accuracy)

    all_mrr_scores.append(mrr_score)
    all_action_accuracies.append(action_accuracy)

    # Check if this is the best model
    if final_score > best_score:
        best_score = final_score
        best_model = model  # Keep best model in memory

    # Print Results
    print(f"  🔹 Fold {fold+1} Results:")
    print(f"     - MRR Score: {mrr_score:.4f}")
    print(f"     - Action Accuracy: {action_accuracy:.4f}")
    print(f"     - Final Score: {final_score:.4f}")

# ---------------------------
# Final Cross-Validation Scores
# ---------------------------
avg_mrr = sum(all_mrr_scores) / len(all_mrr_scores)
avg_action_acc = sum(all_action_accuracies) / len(all_action_accuracies)
final_cv_score = (0.7 * avg_mrr) + (0.3 * avg_action_acc)

print("\n Final Cross-Validation Results:")
print(f"  🔹 Average MRR Score: {avg_mrr:.4f}")
print(f"  🔹 Average Action Accuracy: {avg_action_acc:.4f}")
print(f"  🔹 Final Score (70% MRR + 30% Accuracy): {final_cv_score:.4f}")


 Fold 1 of Cross-Validation
  🔹 Fold 1 Results:
     - MRR Score: 0.0006
     - Action Accuracy: 0.9723
     - Final Score: 0.2921

 Fold 2 of Cross-Validation
  🔹 Fold 2 Results:
     - MRR Score: 0.0006
     - Action Accuracy: 0.9846
     - Final Score: 0.2958

 Fold 3 of Cross-Validation
  🔹 Fold 3 Results:
     - MRR Score: 0.0006
     - Action Accuracy: 0.9959
     - Final Score: 0.2992

 Fold 4 of Cross-Validation
  🔹 Fold 4 Results:
     - MRR Score: 0.0006
     - Action Accuracy: 0.9537
     - Final Score: 0.2865

 Fold 5 of Cross-Validation
  🔹 Fold 5 Results:
     - MRR Score: 0.0006
     - Action Accuracy: 0.9185
     - Final Score: 0.2759

 Final Cross-Validation Results:
  🔹 Average MRR Score: 0.0006
  🔹 Average Action Accuracy: 0.9650
  🔹 Final Score (70% MRR + 30% Accuracy): 0.2899


## Cross Validation - Measuring Model Performance

In [ ]:
def cross_validation(X, y, n_folds=5):
    mrr_validation_error = []
    action_validation_error = []

    # split dataset
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
        print(f"Fold {fold+1}:")

        # Split the data
        x_train, x_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Expanding window on the session dataset
        x_train, y_train = expand_dataset(x_train, y_train)
        x_val, y_val = expand_dataset(x_val, y_val)

        # Run the model on (x_train, y_train)

        # Evaluate the model on (x_val, y_val)
        mrr = 0
        action_accuracy = 0


        # Save the Fold evaluation results
        mrr_validation_error.append(mrr)
        action_validation_error.append(action_accuracy)

    print(mrr_validation_error)
    print(action_validation_error)
    return mrr_validation_error, action_validation_error

# cross_validation(df_x, df_y)

In [ ]:
import ast
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD

# Create a temporary ordering for all jobs using the keys from job_embeddings
features_list = list(job_embeddings.keys())
num_features = len(features_list)  # Number of features for which we have embeddings
pca_dim = 50  # Choose number of PCA components

# Apply PCA separately to each feature embedding
reduced_embeddings = {}
for feature in features_list:
    pca_model = PCA(n_components=pca_dim)
    feature_emb_np = job_embeddings[feature].cpu().numpy()
    feature_emb_pca_np = pca_model.fit_transform(feature_emb_np)
    reduced_embeddings[feature] = torch.tensor(feature_emb_pca_np, dtype=torch.float32, device=device)

# Concatenate the reduced feature embeddings
job_emb_matrix_pca = torch.cat([reduced_embeddings[f] for f in features_list], dim=1)
# job_emb_matrix_pca: (num_jobs, pca_dim * num_features)

In [ ]:
print(job_embedding_matrix.shape, job_emb_matrix_pca.shape)

torch.Size([21917, 5120]) torch.Size([21917, 250])


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=64): # add hidden_dim
        super(MLP, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),  # input_dim is now 250 (from pca_dim * num_features)
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim) # connecting hidden to output layer
        )

    def forward(self, x):
        return self.fc(x)

# Create the MLP model
input_dim = job_emb_matrix_pca.shape[1] # matrix of shape (num_jobs, num_features*emb_dim_after_pca)

model_mlp = MLP(input_dim=input_dim).to(device)

In [ ]:
# df_x.loc[:,"job_ids"] = df_x["job_ids"].apply(ast.literal_eval)  # e.g., [102, 203, ...]
# df_x.loc[:,"actions"] = df_x["actions"].apply(ast.literal_eval)  # e.g., [1, 2, ...]
df_x.loc[:,"job_ids"] = df_x["job_ids"].apply(lambda jobs: [int(j) for j in jobs])  # e.g., [102, 203, ...]
df_x.loc[:,"actions"] = df_x["actions"].apply(lambda actions: [action_weights[a] for a in actions])  # e.g., [1, 2, ...]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd

def train(model, x, y, job_emb_matrix_pca, num_epochs=50, lr=0.001, eval_every=10):
    model.train()

    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()  # Loss function to optimize ranking

    for epoch in range(num_epochs):
        optimizer.zero_grad()

        # Compute transformed embeddings for all jobs using the MLP model
        batch_size = 64  # Adjust this size to fit the GPU memory
        transformed_embeddings = []
        for i in range(0, job_emb_matrix_pca.shape[0], batch_size):
            batch = job_emb_matrix_pca[i:i + batch_size].to(device).float()  # Move batch to GPU and ensure it's float32
            with torch.no_grad():  # No need to compute gradients for inference
                transformed_embeddings.append(model(batch))  # Pass batch through the model
            del batch  # Explicitly delete to free up memory after use

        # Concatenate all embeddings after batch processing
        transformed_embeddings = torch.cat(transformed_embeddings, dim=0)  # Shape: (num_jobs, emb_dim)

        torch.cuda.empty_cache()  # Clear GPU cache

        total_loss = 0

        # Loop over each session in x
        for i in range(len(x)):
            session_job_ids = x.iloc[i]["job_ids"]   # e.g., [101, 205, 309, ...]
            session_actions = torch.tensor(x.iloc[i]["actions"], dtype=torch.float32, device=device)

            session_embeddings = transformed_embeddings[session_job_ids]  # shape: (num_jobs_in_session, emb_dim)
            weights = session_actions.view(-1, 1)  # Reshape to match (num_jobs, 1)
            session_profile = (session_embeddings * weights).sum(dim=0) / weights.sum()  # Weighted mean
            session_profile = session_profile.requires_grad_()

            # Cosine similarity
            batch_size_similarity = 128  # Adjust batch size based on GPU memory
            similarities = torch.zeros(len(transformed_embeddings), device=device)
            with torch.no_grad():
                for j in range(0, len(transformed_embeddings), batch_size_similarity):
                    batch = transformed_embeddings[j:j + batch_size_similarity]
                    similarities[j:j + batch_size_similarity] = F.cosine_similarity(session_profile.unsqueeze(0), batch)

            # Rank the jobs by similarity
            similarities = similarities.requires_grad_()

            ranked_jobs = torch.argsort(similarities, descending=True)

            # Compute loss - Encourage true job to rank high
            y_true = int(y.iloc[i]["job_id"])
            y_true = torch.tensor([y_true], dtype=torch.long, device=device)  # Convert job ID to index

            # CrossEntropyLoss requires the predictions to be raw scores (logits), not probabilities
            loss = loss_fn(similarities.unsqueeze(0), y_true)

            total_loss += loss.item()
            loss.backward(retain_graph=True)

            del session_embeddings  # Delete session_embeddings after use
            del session_actions     # Delete session_actions after use
            torch.cuda.empty_cache()  # Clear GPU cache after each session

        optimizer.step()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(x):.4f}")

    return model


In [ ]:
# split dataset
kf = KFold(n_splits=2, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(kf.split(df_x, df_y)):
    print(f"Fold {fold+1}:")

    # Split the data
    x_train, x_val = df_x.iloc[train_index], df_x.iloc[val_index]
    y_train, y_val = df_y.iloc[train_index], df_y.iloc[val_index]

    train(model_mlp, x_train, y_train, job_emb_matrix_pca, num_epochs=50, lr=0.001, eval_every=10)

    # y_pred = model(x_val)

    # mrr_score = mean_reciprocal_rank(y_true_jobs, y_pred_jobs)


Fold 1:


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# # Training loop for the MLP model using the CBF score as supervision

# def train(model, x, y, num_epochs=50, lr=0.001, eval_every=10):
#     model.train()

#     optimizer_mlp = optim.Adam(model_mlp.parameters(), lr=0.001)
#     criterion = nn.CrossEntropyLoss()

#     for epoch in range(num_epochs):
#         optimizer_mlp.zero_grad()

#         # Compute transformed embeddings for all jobs using the current MLP model
#         transformed_embeddings = model_mlp(job_emb_matrix_pca)  # shape: (num_jobs, raw_emb_dim)
#         aligned_embeddings = build_aligned_embeddings(job2idx, transformed_embeddings)

#         # Loop over each session in x
#         for i in range(len(x)):
#             # Retrieve the session's job_ids and actions
#             session_job_ids = x.iloc[i]["job_ids"]     # e.g., [101, 205, 309, ...]
#             session_actions = x.iloc[i]["actions"]       # e.g., [1, 2, 1, ...]

#             # Build a session profile as a weighted average of the job embeddings
#             session_profile = torch.zeros(raw_emb_dim, device=device)
#             weight_sum = 0.0
#             for job, action in zip(session_job_ids, session_actions):
#                 if job in temp_job2idx:
#                     idx = temp_job2idx[job]
#                     session_profile += transformed_embeddings[idx] * action
#                     weight_sum += action
#             if weight_sum > 0:
#                 session_profile /= weight_sum
#             else:
#                 session_profile = transformed_embeddings.mean(dim=0)

#             # Compute CBF scores as dot products between the session profile and all job embeddings
#             scores = torch.matmul(transformed_embeddings, session_profile)  # shape: (num_jobs,)

#             # Retrieve the target job for the session from df_y (assumed column "target_job")
#             target_job = y.iloc[i]["job_id"]
#             if target_job not in temp_job2idx:
#                 continue  # Skip if target job not found
#             target_idx = temp_job2idx[target_job]

#             # Prepare scores and target for cross-entropy loss (add batch dimension)
#             scores = scores.unsqueeze(0)  # now shape: (1, num_jobs)
#             target_tensor = torch.tensor([target_idx], dtype=torch.long, device=device)

#             loss = criterion(scores, target_tensor)
#             total_loss += loss
#             total_loss = total_loss / valid_sessions
#             total_loss.backward()
#             optimizer_mlp.step()
#             print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item():.4f}")

# # After training, update the job embedding matrix (to be used later in the pipeline)
# model_mlp.eval()
# with torch.no_grad():
#     job_embedding_matrix = model_mlp(job_emb_matrix_pca)
# print("MLP training complete. New job_embedding_matrix shape:", job_embedding_matrix.shape)

In [ ]:
results = {}
y_true_jobs = []
y_pred_jobs = []
y_true_actions = []
y_pred_actions = []
idx2job = {i: j for j, i in job2idx.items()}

for _, row in df_y.iterrows():
    s_id = row['session_id']
    if s_id in session2idx:
        s_idx = session2idx[s_id]

        interacted_jobs = np.where(interaction_matrix[s_idx] > 0)[0]
        if len(interacted_jobs) > 0:
            session_embeddings = aligned_embeddings[interacted_jobs]
            action_weights = torch.tensor(interaction_matrix[s_idx, interacted_jobs], device=device).float()
            action_weights = action_weights.view(-1, 1)
            user_profile_vector = torch.sum(session_embeddings * action_weights, dim=0) / torch.sum(action_weights)
        else:
            user_profile_vector = aligned_embeddings.mean(dim=0)

        cbf_scores = user_profile_vector @ aligned_embeddings.T

# Result test:

In [ ]:
import pandas as pd
import torch
import numpy as np

df_challenge = pd.read_csv('/content/x_test_jCBBNP2.csv')
df_challenge["job_ids"] = df_challenge["job_ids"].apply(ast.literal_eval)
df_challenge["actions"] = df_challenge["actions"].apply(ast.literal_eval)

# (Assuming each row in df_x has a list of actions corresponding to the jobs in that session)
df_challenge['apply_count'] = df_challenge['actions'].apply(lambda acts: sum(1 for act in acts if act == 'apply'))
df_challenge['view_count'] = df_challenge['actions'].apply(lambda acts: sum(1 for act in acts if act == 'view'))
df_challenge['total'] = df_challenge['apply_count'] + df_challenge['view_count']

# ---------------------------
# Run Predictions on x_challenge
# ---------------------------
results = []
idx2job = {i: j for j, i in job2idx.items()}

for _, row in df_challenge.iterrows():
    s_id = row['session_id']

    if s_id in session2idx:
        s_idx = session2idx[s_id]

        # Get top-10 job recommendations
        top_indices = hybrid_recommendation(s_idx, user_factors, job_factors, interaction_matrix, aligned_embeddings, alpha=0.75, top_n=10)
        recommended_jobs = [idx2job[i] for i in top_indices]
    else:
        recommended_jobs = []  # No interaction history, fallback to empty

    # Predict action using trained MLP
    feat = row[['total', 'apply_count', 'view_count']].astype(float).values.reshape(1, -1)
    feat = torch.tensor(feat, dtype=torch.float32, device=device)

    predicted_action = 'apply' if best_model(feat).item() > 0.5 else 'view'

    # Store result
    results.append([s_id, predicted_action, str(recommended_jobs)])

# ---------------------------
# Save Results to CSV
# ---------------------------
df_results = pd.DataFrame(results, columns=['session_id', 'action', 'job_id'])
df_results.to_csv('y_challenge_mlp.csv', index=False)